# Test update instance

In [ ]:
from laminhub_rest.core.account import delete_account
from laminhub_rest.core.instance import update_instance, delete_instance
from laminhub_rest.core.instance._crud import sb_select_instance
from laminhub_rest.utils._test import (
    create_test_auth,
    create_test_account,
    create_test_instance,
    create_test_storage,
    add_test_collaborator,
)
from laminhub_rest.connector import connect_hub_with_auth
from laminhub_rest.main import client
from laminhub_rest._clean_ci import clean_ci

## Parametrize 

In [ ]:
auth = create_test_auth()
access_token = auth["access_token"]
account = create_test_account(handle=auth["handle"], access_token=access_token)
storage = create_test_storage(access_token=access_token)
instance = create_test_instance(storage_id=storage["id"], access_token=access_token)

instance_id = instance["id"]

## Add a collaborator

In [ ]:
response = client.post(
    f"/instance/resources/accounts/?handle={account['handle']}&instance_owner_handle={account['handle']}&instance_name={instance['name']}&role=admin",
    headers={"authentication": f"Bearer {access_token}"},
).json()

assert response == "success"

## Update an instance

### Call `update_instance`

In [ ]:
new_description_1 = "Description"

In [ ]:
message = update_instance(
    instance_id=instance_id,
    description=new_description_1,
    _access_token=auth["access_token"],
)

In [ ]:
hub = connect_hub_with_auth(access_token=auth["access_token"])
instance = sb_select_instance(instance_id, hub)

assert message is None
assert instance["description"] == new_description_1

### Put method on `/instance` route

In [ ]:
new_description_2 = "Description 2"

In [ ]:
response = client.put(
    f"/instance/?instance_id={instance_id}&description={new_description_2}",
    headers={"authentication": f"Bearer {access_token}"},
)

In [ ]:
instance = sb_select_instance(instance_id, hub)

assert response.json() == "success"
assert instance["description"] == new_description_2

## Update a non existing instance

### Call `update_instance` on a non existing instance

In [ ]:
non_existing_instance_id = instance_id[:-5] + "1aa63"

In [ ]:
message = update_instance(
    instance_id=non_existing_instance_id,
    description="Description",
    _access_token=auth["access_token"],
)

In [ ]:
assert message == "instance-not-updated"

### Put method on `/instance` route

In [ ]:
response = client.put(
    f"/instance/?instance_id={non_existing_instance_id}&description={new_description_2}",
    headers={"authentication": f"Bearer {access_token}"},
)

In [ ]:
assert response.json() == "instance-not-updated"

## Clean up test assets

In [ ]:
delete_account(account["handle"], access_token)
delete_instance(
    owner=account["handle"], name=instance["name"], _access_token=access_token
)

In [ ]:
clean_ci()